---
title: "End-To-End Example with Scikit-Learn, Hops and the Feature Store - Iris Flower Classification"
date: 2021-02-24
type: technical_note
draft: false
---

# End-To-End Example with Scikit-Learn, HSML and the Feature Store - Iris Flower Classification
---
*FEATURE ENGINEERING --> MODEL TRAINING --> MODEL SERVING --> MODEL MONITORING*


In this notebook we will, 

1. Load the Iris Flower dataset from HopsFS
2. Do feature engineering on the dataset
3. Save the features to the feature store
4. Read the feature data from the feature store
5. Train a KNN Model using SkLearn
6. Save the trained model to HopsFS
7. Launch a serving instance to serve the trained model
8. Send some prediction requests to the served model
9. Monitor the predictions through Kafka

## Model Serving on [Hopsworks](https://github.com/logicalclocks/hopsworks)

![hops.png](../../images/hops.png)

## The `hops` python library

`hops` is a helper library for Hops that facilitates development by hiding the complexity of running applications and iteracting with services.

Have a feature request or encountered an issue? Please let us know on <a href="https://github.com/logicalclocks/hops-util-py">github</a>.

### Using the `experiment` module

To be able to run your Machine Learning code in Hopsworks, the code for the whole program needs to be provided and put inside a wrapper function. Everything, from importing libraries to reading data and defining the model and running the program needs to be put inside a wrapper function.

The `experiment` module provides an api to Python programs such as TensorFlow, Keras and PyTorch on a Hopsworks on any number of machines and GPUs.

An Experiment could be a single Python program, which we refer to as an **Experiment**. 

Grid search or genetic hyperparameter optimization such as differential evolution which runs several Experiments in parallel, which we refer to as **Parallel Experiment**. 

ParameterServerStrategy, CollectiveAllReduceStrategy and MultiworkerMirroredStrategy making multi-machine/multi-gpu training as simple as invoking a function for orchestration. This mode is referred to as **Distributed Training**.

### Using the `hdfs` module
The `hdfs` module provides a method to get the path in HopsFS where your data is stored, namely by calling `hdfs.project_path()`. The path resolves to the root path for your project, which is the view that you see when you click `Data Sets` in HopsWorks. To point where your actual data resides in the project you to append the full path from there to your Dataset. For example if you create a mnist folder in your Resources Dataset, the path to the mnist data would be `hdfs.project_path() + 'Resources/mnist'`

```python
# Use this module to get the path to your project in HopsFS, then append the path to your Dataset in your project
from hops import hdfs
project_path = hdfs.project_path()
```

```python
from hops import hdfs
# Uploading the traied model to hdfs
hdfs.copy_to_hdfs("iris_knn.pkl", "Resources/iris_flower", overwrite=True)

# Downloading the iris flower model to the current working directory
iris_flower_model_hdfs_path = hdfs.project_path() + "Resources/iris_flower/iris_knn.pkl"
local_iris_flower_model_path = hdfs.copy_to_local(iris_flower_model_hdfs_path)
```

### Documentation
See the following links to learn more about running experiments in Hopsworks

- <a href="https://hopsworks.readthedocs.io/en/latest/hopsml/experiment.html">Learn more about experiments</a>
<br>
- <a href="https://hopsworks.readthedocs.io/en/latest/hopsml/hopsML.html">Building end-to-end pipelines</a>
<br>
- Give us a star, create an issue or a feature request on  <a href="https://github.com/logicalclocks/hopsworks">Hopsworks github</a>

### Managing experiments
Experiments service provides a unified view of all the experiments run using the `experiment` module.
<br>
As demonstrated in the gif it provides general information about the experiment and the resulting metric. Experiments can be visualized meanwhile or after training in a TensorBoard.
<br>
<br>
![Image7-Monitor.png](../../images/experiments.gif)

## Import libraries

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import joblib
from pyspark.ml.feature import StringIndexer
from pyspark.sql.types import IntegerType
import numpy as np
import time
import json
from hops import kafka, hdfs, tls
from confluent_kafka import Producer, Consumer, KafkaError
import random

Starting Spark application


ID,Application ID,Kind,State,Spark UI,Driver log
0,application_1652804119304_0001,pyspark,idle,Link,Link


SparkSession available as 'spark'.


## Prepare Training Dataset

### Load Iris Dataset (csv)

In [2]:
project_path = hdfs.project_path()
iris_df = spark.read.format("csv").option("header", "true").option("inferSchema", True).load(
    project_path + "TourData/iris/iris.csv")

In [3]:
iris_df.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- variety: string (nullable = true)

### Feature  Engineering

The dataset is already quite well prepared, the only thing we need to for feature engineering is to convert the `variety` column to numeric and save a lookup table so that we later on can convert the numeric representation back to the categorical representation.

In [4]:
encoder = StringIndexer(inputCol="variety", outputCol="label")
fit_model = encoder.fit(iris_df)
iris_df1 = fit_model.transform(iris_df)
lookup_df = iris_df1.select(["variety", "label"]).distinct()
iris_df2 = iris_df1.drop("variety")
iris_df3 = iris_df2.withColumn("label", iris_df2["label"].cast(IntegerType()))
iris_df3.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- label: integer (nullable = true)

In [5]:
iris_df3.show(5)

+------------+-----------+------------+-----------+-----+
|sepal_length|sepal_width|petal_length|petal_width|label|
+------------+-----------+------------+-----------+-----+
|         5.1|        3.5|         1.4|        0.2|    0|
|         4.9|        3.0|         1.4|        0.2|    0|
|         4.7|        3.2|         1.3|        0.2|    0|
|         4.6|        3.1|         1.5|        0.2|    0|
|         5.0|        3.6|         1.4|        0.2|    0|
+------------+-----------+------------+-----------+-----+
only showing top 5 rows

In [6]:
lookup_df.show(3)

+----------+-----+
|   variety|label|
+----------+-----+
| Virginica|  2.0|
|Versicolor|  1.0|
|    Setosa|  0.0|
+----------+-----+

### Save Features to the Feature Store

We can save two feature groups (hive tables), one called `iris_features` that contains the iris features and the corresponding numeric label, and another feature group called `iris_labels_lookup` for converting the numeric iris label back to categorical.

**Note**: To be able to run the feature store code, you first have to enable the Feature Store Service in your project. To do this, go to the "Settings" tab in your project, select the feature store service and click "Save". 

In [7]:
import hsfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [8]:
iris_features = fs.create_feature_group(name="iris_features", version=1, time_travel_format=None)
iris_features.save(iris_df3)

In [9]:
iris_labels_lookup = fs.create_feature_group(name="iris_labels_lookup", version=1, time_travel_format=None)
iris_labels_lookup.save(lookup_df)

### Read the Iris Training Dataset from the Feature Store

In [10]:
iris_features = fs.get_feature_group("iris_features", 1)
df = iris_features.read().toPandas()
df.head(10)

   sepal_length  sepal_width  petal_length  petal_width  label
0           5.1          3.5           1.4          0.2      0
1           4.9          3.0           1.4          0.2      0
2           4.7          3.2           1.3          0.2      0
3           4.6          3.1           1.5          0.2      0
4           5.0          3.6           1.4          0.2      0
5           5.4          3.9           1.7          0.4      0
6           4.6          3.4           1.4          0.3      0
7           5.0          3.4           1.5          0.2      0
8           4.4          2.9           1.4          0.2      0
9           4.9          3.1           1.5          0.1      0

In [11]:
df.describe()

       sepal_length  sepal_width  petal_length  petal_width       label
count    150.000000   150.000000    150.000000   150.000000  150.000000
mean       5.843333     3.057333      3.758000     1.199333    1.000000
std        0.828066     0.435866      1.765298     0.762238    0.819232
min        4.300000     2.000000      1.000000     0.100000    0.000000
25%        5.100000     2.800000      1.600000     0.300000    0.000000
50%        5.800000     3.000000      4.350000     1.300000    1.000000
75%        6.400000     3.300000      5.100000     1.800000    2.000000
max        7.900000     4.400000      6.900000     2.500000    2.000000

In [12]:
x_df = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y_df = df[["label"]]
X = x_df.values
y = y_df.values.ravel()

## Train an Iris Flower Classifier

In [13]:
IRIS_FLOWER_RESOURCES_PATH = "/Resources/iris_flower"
IRIS_FLOWER_MODEL_PKL = "iris_knn.pkl"

### Generate a KNN Model using the Feature Data

In [14]:
neighbors = random.randint(3, 30)
iris_knn = KNeighborsClassifier(n_neighbors=neighbors)
iris_knn.fit(X, y)
y_pred = iris_knn.predict(X)
acc = accuracy_score(y, y_pred)
print(acc)

0.98

### Save the Trained Model to HopsFS

In [15]:
joblib.dump(iris_knn, IRIS_FLOWER_MODEL_PKL)
hdfs.mkdir(IRIS_FLOWER_RESOURCES_PATH)
hdfs.copy_to_hdfs(IRIS_FLOWER_MODEL_PKL, IRIS_FLOWER_RESOURCES_PATH, overwrite=True)

### Export the Trained Model to the Models Repository

It is a best-practice to put the python script that loads and run the model together with the trained model files. Below is the code for copying the script into the iris flower resources folder, which will be used when exporting the model.

In [16]:
PREDICTOR_SCRIPT_NAME = "iris_flower_classifier.py"
PREDICTOR_SCRIPT_PATH = "Jupyter/end_to_end_pipeline/sklearn/" + PREDICTOR_SCRIPT_NAME

hdfs.cp(PREDICTOR_SCRIPT_PATH, IRIS_FLOWER_RESOURCES_PATH + "/" + PREDICTOR_SCRIPT_NAME, overwrite=True)

In [17]:
import hsml
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

MODEL_NAME = "irisflowerclassifier"

input_schema = Schema(x_df)
output_schema = Schema(y_df)

conn = hsml.connection()
mr = conn.get_model_registry()

tf_model = mr.sklearn.create_model(MODEL_NAME, 
                                   metrics={'accuracy': acc},
                                   input_example=x_df,
                                   model_schema=ModelSchema(input_schema=input_schema, output_schema=output_schema))

tf_model.save(IRIS_FLOWER_RESOURCES_PATH)

Connected. Call `.close()` to terminate connection gracefully.
Exported model irisflowerclassifier with version 1
Model('irisflowerclassifier', 1, 'SKLEARN')
Model export complete: 100%|##########| 6/6 [00:13<00:00,  2.33s/it]                   

In [18]:
for p in hdfs.ls("Models/" + MODEL_NAME, recursive=True):
    print(p)

hdfs://rpc.namenode.service.consul:8020/Projects/demo_ml_meb10000/Models/irisflowerclassifier/1
hdfs://rpc.namenode.service.consul:8020/Projects/demo_ml_meb10000/Models/irisflowerclassifier/1/input_example.json
hdfs://rpc.namenode.service.consul:8020/Projects/demo_ml_meb10000/Models/irisflowerclassifier/1/iris_flower_classifier.py
hdfs://rpc.namenode.service.consul:8020/Projects/demo_ml_meb10000/Models/irisflowerclassifier/1/iris_knn.pkl
hdfs://rpc.namenode.service.consul:8020/Projects/demo_ml_meb10000/Models/irisflowerclassifier/1/model_schema.json
hdfs://rpc.namenode.service.consul:8020/Projects/demo_ml_meb10000/Models/irisflowerclassifier/1/program.ipynb

## Serve the Trained Model

To serve a SkLearn Model, write a python script that downloads the HDFS model in the constructor and saves it as a class variable and then implements the `Predict` class and the methods `predict`, `classify` and `regress`, like this:

```python
from sklearn.externals import joblib
from hops import hdfs

class Predict(object):

    def __init__(self):
        """Prepare and load a trained model"""
        self.model_path = "Models/irisflowerclassifier/1/iris_knn.pkl"
        print("Copying scikit-learn model from HDFS to local directory")
        hdfs.copy_to_local(self.model_path)
        print("Loading local scikit-learn model")
        self.model = joblib.load("./iris_knn.pkl")
        print("Trained model is ready")

    def predict(self, inputs):
        """Serves prediction using a trained model"""
        return self.model.predict(inputs).tolist() # Numpy Arrays are note JSON serializable
```

Then upload this python script to some folder in your project and go to the "Model Serving" service in Hopsworks:

![sklearn_serving1.png](./../../images/sklearn_serving1.png)

Then click on "create serving" and configure your serving:

![sklearn_serving2.png](./../../images/sklearn_serving2.png)

Once the serving is created, you can start it and view information like server-logs and which kafka topic it is logging inference requests to.

![sklearn_serving3.png](./../../images/sklearn_serving3.png)

### Query Model Repository for best IrisFlowerClassifier Model

In [19]:
# Get best version of the iris flower classifier model
best_model = mr.get_best_model(MODEL_NAME, "accuracy", "max")

print('Model name: ' + best_model.name)
print('Model version: ' + str(best_model.version))
print(best_model.training_metrics)

Model name: irisflowerclassifier
Model version: 1
{'accuracy': '0.98'}

### Create a Deployment for the Trained Model

In [21]:
PREDICTOR_SCRIPT_PATH = best_model.version_path + "/" + PREDICTOR_SCRIPT_NAME

# Deploy the best version of the model
irisclassifier = best_model.deploy(script_file=PREDICTOR_SCRIPT_PATH)  # To serve models with KServe, set serving_tool to "KSERVE"

After the serving have been created, you can find it in the Hopsworks UI by going to the "Deployments" tab. You can also use the class attributes or the `.describe()` method of a deployment object to describe access its metadata.

In [22]:
print("Description of " + irisclassifier.name)
irisclassifier.describe()

Description of irisflowerclassifier
{
    "artifact_version": 1,
    "batching_enabled": false,
    "created": "2022-05-18T12:40:06.464Z",
    "creator": "Admin Admin",
    "id": 1,
    "inference_logging": "ALL",
    "kafka_topic_dto": {
        "name": "CREATE",
        "num_of_partitions": 1,
        "num_of_replicas": 1
    },
    "model_name": "irisflowerclassifier",
    "model_path": "/Projects/demo_ml_meb10000/Models/irisflowerclassifier",
    "model_server": "PYTHON",
    "model_version": 1,
    "name": "irisflowerclassifier",
    "predictor": "iris_flower_classifier.py",
    "predictor_resource_config": {
        "cores": 1,
        "gpus": 0,
        "memory": 1024
    },
    "requested_instances": 1,
    "serving_tool": "DEFAULT"
}

## Classify Iris Flowers using the Trained Model

In [23]:
irisclassifier.start()

Deployment is running: 100%|##########| 1/1 [00:05<00:00,  5.49s/it]

### Send Prediction Requests to the Deployed Model

For making inference requests you can use the utility method `.predict()` from the deployment object.

In [26]:
for i in range(10):
    data = {"instances" : [best_model.input_example]}
    predictions = irisclassifier.predict(data)
    print(predictions)

{'predictions': [0]}
{'predictions': [0]}
{'predictions': [0]}
{'predictions': [0]}
{'predictions': [0]}
{'predictions': [0]}
{'predictions': [0]}
{'predictions': [0]}
{'predictions': [0]}
{'predictions': [0]}

## Monitor Prediction Logs

### Consume Prediction Requests and Responses using Kafka

All prediction requestst are automatically logged to Kafka which means that you can keep track for your model's performance and its predictions in a scalable manner.

Setup a Kafka consumer and subscribe to the topic containing the prediction logs

In [27]:
TOPIC_NAME = irisclassifier.inference_logger.kafka_topic.name

config = kafka.get_kafka_default_config()
config['default.topic.config'] = {'auto.offset.reset': 'earliest'}
consumer = Consumer(config)
topics = [TOPIC_NAME]
consumer.subscribe(topics)

Read the Kafka Avro schema from Hopsworks and setup an Avro reader

In [28]:
json_schema = kafka.get_schema(TOPIC_NAME)
avro_schema = kafka.convert_json_schema_to_avro(json_schema)

Read the lookup table from the Feature Store for converting numerical labels to categorical

In [29]:
iris_labels_lookup = fs.get_feature_group("iris_labels_lookup", 1)
iris_labels_lookup_df = iris_labels_lookup.read().toPandas()

Read messages from the Kafka topic, parse them with the Avro schema and print the results

In [30]:
PRINT_INSTANCES=False
PRINT_PREDICTIONS=True

for i in range(0, 5):
    msg = consumer.poll(timeout=5.0)
    if msg is not None:
        value = msg.value()
        try:
            event_dict = kafka.parse_avro_msg(value, avro_schema)
            
            print("serving: {}, version: {}, timestamp: {},\n"\
                  "         http_response_code: {}, model_server: {}, serving_tool: {}".format(
                       event_dict["modelName"],
                       event_dict["modelVersion"],
                       event_dict["requestTimestamp"],
                       event_dict["responseHttpCode"],
                       event_dict["modelServer"],
                       event_dict["servingTool"]))
            
            if PRINT_INSTANCES:
                print("instances: {}\n".format(event_dict["inferenceRequest"]))
            if PRINT_PREDICTIONS:
                prediction = json.loads(event_dict["inferenceResponse"])["predictions"][0]
                prediction_label = iris_labels_lookup_df.loc[iris_labels_lookup_df['label'] == prediction, 'variety'].iloc[0]
                print("predictions: {}, prediction_label:{}\n".format(prediction, prediction_label))

        except Exception as e:
            print("A message was read but there was an error parsing it")
            print(e)
    else:
        print("timeout.. no more messages to read from topic")

serving: irisflowerclassifier, version: 1, timestamp: 1652878029777,
         http_response_code: 200, model_server: PYTHON, serving_tool: DEFAULT
predictions: 0, prediction_label:Setosa

serving: irisflowerclassifier, version: 1, timestamp: 1652878030212,
         http_response_code: 200, model_server: PYTHON, serving_tool: DEFAULT
predictions: 0, prediction_label:Setosa

serving: irisflowerclassifier, version: 1, timestamp: 1652878031009,
         http_response_code: 200, model_server: PYTHON, serving_tool: DEFAULT
predictions: 0, prediction_label:Setosa

serving: irisflowerclassifier, version: 1, timestamp: 1652878031725,
         http_response_code: 200, model_server: PYTHON, serving_tool: DEFAULT
predictions: 0, prediction_label:Setosa

serving: irisflowerclassifier, version: 1, timestamp: 1652878032543,
         http_response_code: 200, model_server: PYTHON, serving_tool: DEFAULT
predictions: 0, prediction_label:Setosa